<a href="https://colab.research.google.com/github/pmy0792/pytorch_tutorial/blob/main/lab_10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [8]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64,128,kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )

    self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob)
    )
    self.fc2=torch.nn.Linear(625,10,bias=True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=self.layer3(out)
    out=out.view(out.size(0),-1)
    out=self.layer4(out)
    out=self.fc2(out)
    return out

In [9]:
model=CNN().to(device)

In [10]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr= learning_rate)

In [11]:
total_batch = len(data_loader)
model.train()   
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


Learning started. It takes sometime.
[Epoch:    1] cost = 0.190742791
[Epoch:    2] cost = 0.0500841625
[Epoch:    3] cost = 0.0368692651
[Epoch:    4] cost = 0.028539706
[Epoch:    5] cost = 0.0227480605
[Epoch:    6] cost = 0.0188842956
[Epoch:    7] cost = 0.016331885
[Epoch:    8] cost = 0.0138549153
[Epoch:    9] cost = 0.0125691108
[Epoch:   10] cost = 0.0101731755
[Epoch:   11] cost = 0.011381207
[Epoch:   12] cost = 0.009272811
[Epoch:   13] cost = 0.00949806347
[Epoch:   14] cost = 0.00740340911
[Epoch:   15] cost = 0.00688308896


In [12]:
with torch.no_grad():
    model.eval() 
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9878000020980835


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
